# Introduction

The pricing model utilises LSV model with calibration, to price autocallables with three underlyings. Since these models often take a long time to train under Monte Carlo simulation, we attempt to price these derivatives with machine learning models in a more swiftly manner.

In this notebook, we would be doing four things:
1. Data preprocessing
2. Building the neural network
3. Explanability
4. Making the predictions

# 1. Data engineering

We first do EDA and find out that 14541000 rows of data are given to us through a generator. Since a large chunk of data is given to us, we cannot use StandardScaler in scikit-learn due to limited memory. Hence, we do the following things:

1. We loop through all 14541000 rows to get mean and standard deviation of each column and paste the values in the cell below.
2. We standard-scale all of the columns (target included).
3. We shuffle each batch of data given to us to ensure a smooth learning curve.

NB The parameter, 'train' in engineerData function is set as True by default when we train our model. It will return the preprocessed inputs and targets. When the function is used to make predictions, 'train' should be set as False to return only the preprocessed inputs.

### Getting the mean and standard deviation of each column

In [ ]:
mean_values = []
std_values = []

# Getting column names of dataset
dataGen = dl.batch(fromRow=1, toRow=14541000)
df = next(dataGen)
columns = df.columns

# Looping through each column 
for col in columns:
    df = pd.Series()
    dataGen = dl.batch(fromRow=1, toRow=14541000)
    
    # Looping through each batch of data
    for _, d in enumerate(dataGen):
        df.append(d[col])
    
    # Appending the mean and standard deviation values to list
    mean_values.append(df.mean())
    std_values.append(df.std())

## Function for data preprocessing

In [ ]:
def engineerData(df, train=True):
    
    import random
    random.seed(10)
    from tensorflow import set_random_seed
    set_random_seed(10)
    
    # Standard Scaling 
    # mean_values, std_values obtained from above 
    df = df.astype(float) 
    for i, j in enumerate(df.columns):
        df[j] = (df[j] - mean_values[i]) / std_values[i]

    # Return inputs and targets if train, else return inputs only
    if train:
        # Shuffle within the dataset
        from sklearn.utils import shuffle
        df = shuffle(df)
        return df.iloc[:,:-1], df.iloc[:,-1]
    else:
        return df

## Model

We use an artificial neural network to model the price. Liu, Oosterlee and Bohte (2018) shows that an aritificial neural network can approximate the Heston stochastic volatility model well with a MSE of 1.65e-8 on test set.

We use an artificial neural network of 10 hidden layers and 100 neurons in each layer since it gives us a smaller bias than models with less layers and more layers will prevent the model from converging and takes too much to time on a 4GB machine.

We use RELU activation, which is a standard activation nowadays and we use He initialization which is a better initialization than Xavier or random initialization when matched with RELU by He, Zhang, Ren and Sun (2015).

We use Adam optimizer which is also a standard optimizer nowadays, and optimize on MSE as it is not practical to optimize on Maximum Absolute Error.


S. Liu, C. Oosterlee, M. Bohte.“Pricing options and computing implied volatilities using neural networks” https://arxiv.org/abs/1901.08943 (2018)

K. He, X. Zhang, S. Ren, J. Sun.“Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification” https://arxiv.org/abs/1502.01852 (2015)

In [ ]:
# Import packages
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
import pandas as pd
import random
from tensorflow import set_random_seed

# Building the neural network
model = Sequential()

for i in range(10):
    random.seed(10)
    set_random_seed(10)
    model.add(Dense(units=100, activation='relu', kernel_initializer="he_uniform"))
model.add(Dense(units=1, activation='linear'))

When the model loss meets the plateau, we use a smaller learning rate to train. The following picture is created by He (2015) and arrows added by Hammel (2019)

K. He, X. Zhang, S. Ren, J. Sun. "Deep Residual Learning for Image Recognition" 
https://arxiv.org/abs/1512.03385 (2015)

B. D. Hammel. 2019. What Learning Rate Should I Use?. online 
Available at: <http://www.bdhammel.com/assets/learning-rate/resnet_loss.png>

In [1]:
# Manually change to 0.0001 and 0.00001 when training loss plateaus
model.compile(loss='mse', optimizer=Adam(lr=0.001)) 

Since there are significant correlations within each batch, we carry out stratified sampling when we train our model. For every epoch, the data drawn from the training set is in different order to ensure more uniform sampling.

1. We first get 1% of data from each batch as validation dataset.
2. We pull out 2.5% of data from each batch without replacement in every epoch and train our neural network with it. (40 times in each epoch)

In [ ]:
# Get the length in each batch
batchlength = []
for d in dataGen:
    batchlength.append(len(d))

# Get the indexes of validation dataset
listofindex = []
for d in batchlength:
    d = [i for i in range(d)]
    random.shuffle(d)
    listofindex.append(d[:int(len(d)*0.01)])
    
# Training for 100 epochs
epochs = 100
splits = 40
for n in range(epoch):
    # Split each batch into 40 portions and get the indexes
    for i, d in enumerate(batchlength):
        d = [a for a in range(d) if a not in listofindex[i]]
        random.shuffle(d)
        batch.append(np.array_split(d, splits))
    
    # Loop for 40 times to train through the whole train dataset as we have limited memory
    for j in range(splits):
        train = pd.DataFrame()
        valid = pd.DataFrame()
        dataGen = dl.batch(fromRow=1, toRow=14541000)
        for i, d in enumerate(dataGen):
            d = d.reset_index(drop=True)
            valid = pd.concat([valid, d.iloc[listofindex[i]]])
            train = pd.concat([train, d.iloc[batch[i][j]]])

        # Transform the data to numpy arrays
        trainX, trainY = engineerData(train)
        trainX = trainX.values
        trainY = trainY.values

        del train

        validX, validY = engineerData(valid)
        validX = validX.values
        validY = validY.values

        model.fit(trainX, trainY, batch_size=64, epochs=1, validation_data=(validX, validY))

        # Free up enough memory for training
        del trainX, trainY, validX, validY

# 3. Explanability


Since there is no built-in attribute of feature importance for a neural network, we will use another approach to solve the problem.

Fisher, Rudin, and Dominici (2018) proposed a method to understand feature importance of different models. 

1. We get the Mean Squared Error of the original model.
2. We shuffle the values within one column and train the model with other columns remain unaltered.
3. We get the Squared Mean Error of the new model and divide it by the original error.
4. We repeat step 2 and 3 until all columns have been shuffled.
5. We rank the error of each model and the model with the higher error indicates that the importance of the feature would be higher.

Due to limited time for training, we would train each column with 5% of the dataset and 2 epochs for illustrative purpose. More data should be trained to show the 'real' feature importance. 

It will take hours to load the following snippet.

A. Fisher, C. Rudin, F. Dominici.“All Models are Wrong, but Many are Useful: Learning a Variable's Importance by Studying an Entire Class of Prediction Models Simultaneously” http://arxiv.org/abs/1801.01489 (2018)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
import pandas as pd
import random
random.seed(10)
from tensorflow import set_random_seed
set_random_seed(10)
import alphien
dl = alphien.data.DataLoader()

# Get names of the columns
dataGen = dl.batch(fromRow=1, toRow=10)
columns = next(dataGen).columns

# Get 5% of the dataset to train
df = pd.DataFrame()
dataGen = dl.batch(fromRow=1, toRow=14541000)
for i, d in enumerate(dataGen):
    d = d.sample(frac=0.05)
    df = pd.concat([df,d])
df = df.reset_index(drop=True)

explanability = []

# Shuffle each column and train the dataset with shuffled columns
for col in columns:
    # Getting the column data under investigation
    dataGen = dl.batch(fromRow=1, toRow=14541000)
    augmented = pd.Series()
    while True:
        try:
            data = next(dataGen)
            augmented = pd.concat([augmented, data[col]])
        except:
            break
            
    augmented = augmented.sample(frac=1)
    
    # Neural network model
    model = Sequential()
    for i in range(10):
        model.add(Dense(units=100, activation='relu', kernel_initializer="he_uniform"))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mse', optimizer='adam')
    
    # Swap the original column with a shuffled column
    df[col] = augmented.sample(n=df.shape[0]).reset_index(drop=True)

    trainX, trainY = engineerData(df)

    trainX = trainX.values
    trainY = trainY.values
    
    history = model.fit(trainX, trainY, batch_size=64, epochs=1)
    explanability.append(list(history.history.values())[0][0])
    
    del trainX, trainY
    
# Show top 10 most important features
explanability = [i - min(explanability) for i in explanability]
values = sorted(explanability)[-11:-1]
highest = sorted(range(len(explanability)), key=lambda i: columns[i])[-11:-1]
from operator import itemgetter
highest = list(itemgetter(*highest)(columns))

# Plot the top 10 important features
import matplotlib.pyplot as plt
plt.title('Top 10 important features')
plt.barh(highest, values)

It is shown that the vov (volatility of variance) of underlying2 will lead to the highest influence to the price, followed by the monthly volatility of underlying2 from 6Y, 5Y, 4Y, etc. with the exception of 6M.

We noticed that for some features, the loss is slightly lower than the original model. 
This shows that it is possible that some features are not useful in predicting the price.

### We also look for a linear relationship between the features and the price through fitting it to a multiple linear regression model.

In [ ]:
import pandas as pd
import alphien
dl = alphien.data.DataLoader()

# Randomly generate 5% data 
df = pd.DataFrame()
dataGen = dl.batch(fromRow=1, toRow=14541000)
for i, d in enumerate(dataGen):
    d = d.sample(frac=0.03)
    df = pd.concat([df,d])

trainX, trainY = engineerData(df)
trainX = trainX.values
trainY = trainY.values
del df
    
    
# Fitting multiple linear regression model
import statsmodels.api as sm

trainX = sm.add_constant(trainX)
model = sm.OLS(trainY, trainX).fit()
model.summary()

# Plotting Top 10 features with highest p values
columns = d.columns
values = sorted(model.pvalues)[-11:-1]
highest = sorted(range(len(model.pvalues)), key=lambda i: columns[i])[-11:-1]
from operator import itemgetter
highest = list(itemgetter(*highest)(columns))

import matplotlib.pyplot as plt
plt.title('Top 10 features with highest p values')
plt.barh(highest, values)

From the summary, we see that the columns are the same with top 10 important features. Since we only trained the neural network for explanability for 10% of data for 1 epoch due to limited time, the similarity in columns may arise due to the fact that the neural network recognises obvious linear relationship in its early stages of training.

None of the columns is statistically significant (p-value > 0.9) to have a linear effect with the price, so the linear relationship between columns and price is inconclusive.

# 4. Making predictions

In [ ]:
def myPredictionFunc(newData, engineerData, model):

    transformed_data = engineerData(newData, train=False) #Apply data transform    
    return model.predict(transformed_data.values) * 0.011214939016011491 + 0.008983811613634216 #Inverse transform scaled target

## Appendix

From the names of the columns, we deduce the followings.


Input variables:

'Undl0volatm_6M' (6-month monthly volatility for underlying asset 0)

'Undl0skew95.105_6M' (6-month for differences in implied volatilities at strikes of 95% and 105% for underlying asset 0)

'Undl0curve95.105_6M' (6-month for differences in curve at strikes of 95% and 105% for underlying asset 0)

'Undl0vov' (Volatility of variance process for underlying asset 0)

'Undl0svc' (Correlation of Wiener processes for asset price and variance for underlying asset 0)

'Undl0mr' (Speed of mean reversion for underlying asset 0)

'ContractFeature_Autocall,BarrierLevel,LevelInitial' (Initial Early redemption level)

'ContractFeature_ExpiryPayment,BarrierLevel' (Barrier for kick-in event)

'ContractFeature_ExpiryPayment,KickInPaymentGearing_ENCODED' (Leverage level upon kick-in event)

'ContractFeature_Schedule,EndDate_ENCODED' (Maturity)

'ContractFeature_Schedule,PeriodFrequency_ENCODED' (Frequency of coupon payments per year)


Target: 

'val_lvsvcharge' (Price)